In [88]:
import numpy as np
import tensorflow as tf

In [513]:
original_data=np.loadtxt('data_new/3.000000_0.800000_40000')

array([ 8.19345 ,  4.03271 ,  2.90642 , 14.0703  , 12.0301  ,  2.3908  ,
       12.8342  ,  2.90601 ,  4.10207 ,  6.58025 ,  2.05338 , 24.384   ,
       10.8333  ,  9.71984 ,  2.75427 ,  3.74869 ,  3.42828 ,  8.58996 ,
       13.22    ,  1.67376 ,  4.94991 ,  4.9963  ,  3.70755 ,  8.01062 ,
        1.78707 ,  6.85165 ,  4.7311  , 11.2409  ,  3.87422 ,  5.27313 ,
        3.13343 ,  7.87588 ,  4.67907 ,  4.03981 , 10.8064  ,  4.2765  ,
        3.02798 ,  3.75984 ,  1.43448 ,  4.38726 ,  9.1278  ,  9.45165 ,
        5.73962 ,  9.23244 ,  2.54559 ,  0.99286 ,  2.00006 ,  7.27741 ,
        3.59924 ,  6.07624 ,  7.36122 ,  8.68763 ,  7.25934 ,  5.49142 ,
        4.10097 , 14.9694  ,  1.92011 ,  3.76524 , 10.3292  ,  9.54717 ,
        6.43856 ,  9.08602 ,  2.61682 ,  3.64464 ,  3.22027 ,  4.12645 ,
        2.77016 , 13.6233  ,  2.89851 ,  5.52759 ,  6.40797 ,  7.56868 ,
        7.68828 ,  0.810673,  3.79652 ,  1.33544 ,  4.99296 ,  2.73193 ,
        9.98885 ,  6.95808 ,  2.81419 ,  6.24503 , 

In [493]:

original_dim = 1  # Single feature
intermediate_dim = 60
latent_dim = 3

# Encoder
encoder_inputs = tf.keras.Input(shape=(original_dim,))
h = tf.keras.layers.Dense(intermediate_dim, activation='relu')(encoder_inputs)
z_mean = tf.keras.layers.Dense(latent_dim)(h)
z_log_sigma = tf.keras.layers.Dense(latent_dim)(h)

# Sampling function
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(z_log_sigma) * epsilon

z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_sigma])

# Decoder
decoder_h = tf.keras.layers.Dense(intermediate_dim, activation='relu')
decoder_mean = tf.keras.layers.Dense(original_dim)
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# VAE model
vae = tf.keras.Model(encoder_inputs, x_decoded_mean)

# VAE loss
xent_loss = tf.keras.losses.mean_squared_error(encoder_inputs, x_decoded_mean)
kl_loss = - 0.5 * tf.keras.backend.mean(1 + z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma), axis=-1)
vae_loss = xent_loss + kl_loss

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

run_data=original_data[0:300]
vae.fit(run_data, epochs=60, batch_size=10)

Epoch 1/60
30/30 [==============================] - 1s 936us/step - loss: 31.3267
Epoch 2/60
30/30 [==============================] - 0s 512us/step - loss: 15.8489
Epoch 3/60
30/30 [==============================] - 0s 549us/step - loss: 11.3416
Epoch 4/60
30/30 [==============================] - 0s 496us/step - loss: 5.9555
Epoch 5/60
30/30 [==============================] - 0s 457us/step - loss: 4.9233
Epoch 6/60
30/30 [==============================] - 0s 460us/step - loss: 4.3090
Epoch 7/60
30/30 [==============================] - 0s 438us/step - loss: 3.8678
Epoch 8/60
30/30 [==============================] - 0s 409us/step - loss: 3.7981
Epoch 9/60
30/30 [==============================] - 0s 483us/step - loss: 3.0989
Epoch 10/60
30/30 [==============================] - 0s 433us/step - loss: 2.9252
Epoch 11/60
30/30 [==============================] - 0s 2ms/step - loss: 2.6307
Epoch 12/60
30/30 [==============================] - 0s 476us/step - loss: 2.5832
Epoch 13/60
30/30 [=====

In [512]:
n_samples = 1000  # Number of data points you want to generate
sampled_z = np.random.normal(size=(n_samples, latent_dim))

# Generate data by decoding the sampled points
decoder_input = tf.keras.Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = tf.keras.Model(decoder_input, _x_decoded_mean)
generated_data = abs(generator.predict(sampled_z))

print(generated_data.var())
print(generated_data.mean())
print(original_data.var())
print(original_data.mean())


32/32 [==============================] - 0s 335us/step
10.78119
3.60257
13.028019805398646
5.716824469178082


In [491]:

#store the generated data
#round the generated data to 5 decimal places
generated_data=generated_data.round(5)
np.savetxt('test_data/generated_data',generated_data.round(5),delimiter='\n')


In [ ]:
#draw
# Importing necessary libraries
import pandas as pd

# Load the first dataset
try:
    dataset1 = pd.read_csv('/mnt/data/3.000000_0.800000_10000', header=None)
    dataset1.columns = ['Value']
except Exception as e:
    dataset1_error = str(e)

# Load the second dataset (generated_data)
try:
    dataset2 = pd.read_csv('/mnt/data/generated_data', header=None)
    dataset2.columns = ['Value']
except Exception as e:
    dataset2_error = str(e)

dataset1.head(), dataset2.head(), dataset1_error if 'dataset1_error' in locals() else None, dataset2_error if 'dataset2_error' in locals() else None
